<a href="https://colab.research.google.com/github/JoaoAiress/College_Projects/blob/main/restaurante_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#Eduardo Pavelecini Belleboni
#João Pedro Siqueira Aires

import threading as th
import time

#definition of the classes
class garcon(th.Thread):
    def __init__(rest, id, restaurant):
        super().__init__()
        rest.restaurant = restaurant
        rest.id = id
    def run(rest):
        for i in range(10):
            with rest.restaurant.condicao:
                while len(rest.restaurant.requests) == rest.restaurant.max_requests:
                    rest.restaurant.condicao.wait()
                pedido = (rest.id, i)
                rest.restaurant.requests.append(pedido)
                rest.restaurant.all_requests += 1
                print(f'garcon {rest.id} recebeu o pedido {i}\n')
                rest.restaurant.condicao.notify_all()
            time.sleep(3.2)

class Cozinheiro(th.Thread):
    def __init__(rest, id, restaurant):
        super().__init__()
        rest.restaurant = restaurant
        rest.id = id
  
    def run(rest):
        while True:
            with rest.restaurant.condicao:
                while not rest.restaurant.requests:
                    rest.restaurant.condicao.wait()
                if rest.restaurant.all_requests == 0:
                    break
                pedido = rest.restaurant.requests.pop(0)
                rest.restaurant.all_requests += 1
                print(f'o cozinheiro {rest.id} preparou o pedido {pedido[1]} do garcon {pedido[0]}')
                rest.restaurant.condicao.notify_all()
            if rest.restaurant.all_requests == 0:
                break
            time.sleep(3)

#definition of restaurant and yours actions
class restaurant:
    def __init__(rest, num_garcons, num_cozinheiros):
        
        rest.condicao = th.Condition()
        rest.garcons = [garcon(1, rest) for i in range(num_garcons)]
        rest.cozinheiros = [Cozinheiro(i, rest) for i in range(num_cozinheiros)]
        rest.requests = []
        rest.all_requests = 0
        rest.max_requests = 4

    def open_rest(rest):
        for cozinheiro in rest.cozinheiros:
            cozinheiro.start()
        for garcon in rest.garcons:
            garcon.start()
        
    def close_rest(rest):
        for garcon in rest.garcons:
            garcon.join()
        with rest.condicao:
            rest.condicao.notify_all()
        for cozinheiro in rest.cozinheiros:
            cozinheiro.join(timeout=4)
            if cozinheiro.is_alive():
                break

#definir o tempo e memoria que necessitou
def medir_tempo(function):
  start = time.time()
  function.open_rest()
  function.close_rest()
  end = time.time()
  tempo = end - start
  print("-"*30)
  print(f"\nO tempo decorrido foi de {tempo} segundos\n")
  print("-"*30)

#"execute area"
if __name__ == "__main__":
    restaurant_instance = restaurant(num_garcons=5, num_cozinheiros=1)
    medir_tempo(restaurant_instance)
    print("-"*30)
    print("\n== Mais um cozinheiro foi adicionado ==\n")
    print("-"*30)
    restaurant_instance = restaurant(num_garcons=5, num_cozinheiros=2)
    medir_tempo(restaurant_instance)

garcon 1 recebeu o pedido 0

o cozinheiro 0 preparou o pedido 0 do garcon 1
garcon 1 recebeu o pedido 0

garcon 1 recebeu o pedido 0

garcon 1 recebeu o pedido 0

garcon 1 recebeu o pedido 0

o cozinheiro 0 preparou o pedido 0 do garcon 1
garcon 1 recebeu o pedido 1

o cozinheiro 0 preparou o pedido 0 do garcon 1
garcon 1 recebeu o pedido 1

o cozinheiro 0 preparou o pedido 0 do garcon 1
garcon 1 recebeu o pedido 2

o cozinheiro 0 preparou o pedido 0 do garcon 1
garcon 1 recebeu o pedido 1

o cozinheiro 0 preparou o pedido 1 do garcon 1
garcon 1 recebeu o pedido 2

o cozinheiro 0 preparou o pedido 1 do garcon 1
garcon 1 recebeu o pedido 3

o cozinheiro 0 preparou o pedido 2 do garcon 1
garcon 1 recebeu o pedido 3

o cozinheiro 0 preparou o pedido 1 do garcon 1
garcon 1 recebeu o pedido 1

o cozinheiro 0 preparou o pedido 2 do garcon 1
garcon 1 recebeu o pedido 4

o cozinheiro 0 preparou o pedido 3 do garcon 1
garcon 1 recebeu o pedido 2

o cozinheiro 0 preparou o pedido 3 do garcon 1
g